# Hierarchical Naive Bayes Model

In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import ExpectationMaximization
import pandas as pd
import random

In [8]:
# Create the Hierarchical Naive Bayes model with latent structure
model = BayesianNetwork(
    ebunch=[
        ('Identity Match', 'Name Match'),
        ('Identity Match', 'DOB Match'),
        ('Identity Match', 'Address Match'),
        ('Identity Match', 'ID Match'),
        ('Name Match', 'First Name Match'),
        ('Name Match', 'Middle Name Match'),
        ('Name Match', 'Last Name Match'),
        ('DOB Match', 'Date of Birth Similarity'),
        ('Address Match', 'Address Similarity'),
        ('ID Match', 'ID Similarity'),
        ('First Name Match', 'First Name Similarity'),
        ('Middle Name Match', 'Middle Name Similarity'),
        ('Last Name Match', 'Last Name Similarity')
    ],
    latents=['Name Match', 'DOB Match', 'Address Match', 'ID Match', 'First Name Match', 'Middle Name Match', 'Last Name Match']
)

In [20]:
# Load training data from csv
sampled_data = pd.read_csv('data/sampleunlabeledident.csv')

# E Step for the Unsupervised Learning: Adds an Identity Match column and assigns either 0 or 1 to each row
sampled_data['Identity Match'] = sampled_data.apply(lambda row: random.randint(0, 1), axis=1)

sampled_data = sampled_data.loc[:,['Identity Match','Date of Birth Similarity','Address Similarity','ID Similarity','First Name Similarity','Middle Name Similarity','Last Name Similarity',]]
sampled_data.head()

,Identity Match,Date of Birth Similarity,Address Similarity,ID Similarity,First Name Similarity,Middle Name Similarity,Last Name Similarity
0,1,3.0,342.0,1.0,6.0,25.0,12.0
1,1,3.0,235.0,0.0,15.0,0.0,14.0
2,0,0.0,16.0,0.0,0.0,25.0,0.0
3,0,2.0,222.0,2.0,15.0,0.0,12.0
4,0,2.0,67.0,4.0,19.0,25.0,0.0


In [26]:
sampled_data.columns

Index(['Identity Match', 'Date of Birth Similarity', 'Address Similarity',
       'ID Similarity', 'First Name Similarity', 'Middle Name Similarity',
       'Last Name Similarity'],
      dtype='object')

In [15]:
model.get_leaves()

['Date Of Birth Similarity',
 'Address Similarity',
 'ID Similarity',
 'First Name Similarity',
 'Middle Name Similarity',
 'Last Name Similarity']

In [13]:
# Fit the model
model.fit(sampled_data, estimator=ExpectationMaximization)

ValueError: variable names of the model must be identical to column names in data